# Team WyldFyrez
## Relation to fire and weather data

blah blah blah stuff

In [19]:
#import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [20]:
#requsting csv and reading
fire_csv = "Resources/fire.csv"

df_fire = pd.read_csv(fire_csv, low_memory=False, dtype={'STAT_CAUSE_DESCR': str,'State': str, 
                                                         'CONT_DATE':str, 'Discovery_Date': str, 
                                                         'FIRE_SIZE':float,'LATITUDE': float,'LONGITUDE':float})

df_fire.head(10)

,Unnamed: 0,OBJECTID,STAT_CAUSE_DESCR,STATE,CONT_DATE,DISCOVERY_DATE,FIRE_SIZE,LATITUDE,LONGITUDE
0,0,1001,Miscellaneous,CA,2005-03-17T00:00:00.000Z,2005-03-17T00:00:00.000Z,0.1,41.211389,-123.655833
1,1,1002,Lightning,OR,2005-07-31T00:00:00.000Z,2005-07-31T00:00:00.000Z,0.5,44.841667,-118.619722
2,2,1003,Campfire,OR,2005-07-31T00:00:00.000Z,2005-07-31T00:00:00.000Z,0.1,44.984444,-118.513611
3,3,1004,Miscellaneous,NM,2005-08-02T00:00:00.000Z,2005-07-31T00:00:00.000Z,1.5,35.751389,-105.651944
4,4,1005,Lightning,UT,2005-06-30T00:00:00.000Z,2005-06-15T00:00:00.000Z,1.5,39.938056,-111.395556
5,5,1006,Miscellaneous,LA,2005-07-02T00:00:00.000Z,2005-07-02T00:00:00.000Z,1.0,30.938889,-93.199722
6,6,1007,Lightning,LA,2005-07-28T00:00:00.000Z,2005-07-28T00:00:00.000Z,1.0,31.258889,-92.764444
7,7,1008,Arson,LA,2005-05-23T00:00:00.000Z,2005-05-23T00:00:00.000Z,1.0,31.631944,-92.511667
8,8,1009,Equipment Use,LA,2005-08-21T00:00:00.000Z,2005-08-21T00:00:00.000Z,15.0,31.066667,-92.958333
9,9,1010,Smoking,KS,2005-08-02T00:00:00.000Z,2005-08-02T00:00:00.000Z,0.5,37.088056,-101.636944


In [21]:
#Cleaning Data frame to remove columns
df_sub = df_fire[['STAT_CAUSE_DESCR', 'STATE', 'CONT_DATE'., 'DISCOVERY_DATE', 'FIRE_SIZE', 'LATITUDE', 'LONGITUDE']]
df_sub.head()

,STAT_CAUSE_DESCR,STATE,CONT_DATE,DISCOVERY_DATE,FIRE_SIZE,LATITUDE,LONGITUDE
0,Miscellaneous,CA,2005-03-17T00:00:00.000Z,2005-03-17T00:00:00.000Z,0.1,41.211389,-123.655833
1,Lightning,OR,2005-07-31T00:00:00.000Z,2005-07-31T00:00:00.000Z,0.5,44.841667,-118.619722
2,Campfire,OR,2005-07-31T00:00:00.000Z,2005-07-31T00:00:00.000Z,0.1,44.984444,-118.513611
3,Miscellaneous,NM,2005-08-02T00:00:00.000Z,2005-07-31T00:00:00.000Z,1.5,35.751389,-105.651944
4,Lightning,UT,2005-06-30T00:00:00.000Z,2005-06-15T00:00:00.000Z,1.5,39.938056,-111.395556


In [22]:
#Droping data that does not have full rows
df_dropna = df_sub.dropna(how="any")
df_dropna.count()

STAT_CAUSE_DESCR    988934
STATE               988934
CONT_DATE           988934
DISCOVERY_DATE      988934
FIRE_SIZE           988934
LATITUDE            988934
LONGITUDE           988934
dtype: int64

In [57]:
date_contained = df_dropna.Date_Contained.tolist()
date_discovery = df_dropna.Discovery_Date.tolist()

In [70]:
date_disc = pd.Series(date_discovery)
start = s.str.split(pat = "T", expand=True)
start_df = pd.DataFrame(end)

start_clean = start_df.rename(columns={ 0: "Date Discovery",1: "Time"})
start_clean.head()

,Date,Time
0,2005-03-17,00:00:00.000Z
1,2005-07-31,00:00:00.000Z
2,2005-07-31,00:00:00.000Z
3,2005-08-02,00:00:00.000Z
4,2005-06-30,00:00:00.000Z


In [71]:
date_cont = pd.Series(date_contained)
end = s.str.split(pat = "T", expand=True)
end_df = pd.DataFrame(start)

end_clean = start_df.rename(columns={ 0 : "Date Contained", 1: "Time"})
end_clean.head()

,Date Contained,Time
0,2005-03-17,00:00:00.000Z
1,2005-07-31,00:00:00.000Z
2,2005-07-31,00:00:00.000Z
3,2005-08-02,00:00:00.000Z
4,2005-06-30,00:00:00.000Z


In [25]:
df_clean = df_dropna.rename(columns={ "STAT_CAUSE_DESCR": "Cause", "STATE": "State", 
                                     "CONT_DATE": "Date_Contained", "DISCOVERY_DATE": "Discovery_Date",
                                     "FIRE_SIZE": "Fire_Size"})
df_clean.head()

,Cause,State,Date_Contained,Discovery_Date,Fire_Size,LATITUDE,LONGITUDE
0,Miscellaneous,CA,2005-03-17T00:00:00.000Z,2005-03-17T00:00:00.000Z,0.1,41.211389,-123.655833
1,Lightning,OR,2005-07-31T00:00:00.000Z,2005-07-31T00:00:00.000Z,0.5,44.841667,-118.619722
2,Campfire,OR,2005-07-31T00:00:00.000Z,2005-07-31T00:00:00.000Z,0.1,44.984444,-118.513611
3,Miscellaneous,NM,2005-08-02T00:00:00.000Z,2005-07-31T00:00:00.000Z,1.5,35.751389,-105.651944
4,Lightning,UT,2005-06-30T00:00:00.000Z,2005-06-15T00:00:00.000Z,1.5,39.938056,-111.395556


In [26]:
print(df_clean.dtypes)

Cause              object
State              object
Date_Contained     object
Discovery_Date     object
Fire_Size         float64
LATITUDE          float64
LONGITUDE         float64
dtype: object
